
<p align="center">
    <img src="https://github.com/GeostatsGuy/GeostatsPy/blob/master/TCG_color_logo.png?raw=true" width="220" height="240" />

</p>

## Subsurface Data Analytics 

## Interactive Demonstration of Machine Learning Model Tuning, Generalization & Overfit

#### Michael Pyrcz, Associate Professor, University of Texas at Austin 

##### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1) | [GeostatsPy](https://github.com/GeostatsGuy/GeostatsPy)

### PGE 383 Exercise: Interactive Predictive Model Complexity Tuning, Generalization & Overfit

Here's a simple workflow, demonstration of predictive machine learning model training and testing for overfit.  We use a:

* simple polynomial model

* 1 preditor feature and 1 response feature

for an high interpretability model/ simple illustration.

#### Train / Test Split

The available data is split into training and testing subsets.

* in general 15-30% of the data is withheld from training to apply as testing data

* testing data selection should be fair, the same difficulty of predictions (offset/different from the training dat

#### Machine Learning Model Traing

The training data is applied to train the model parameters such that the model minimizes mismatch with the training data

* it is common to use **mean square error** (known as a **L2 norm**) as a loss function summarizing the model mismatch

* **miminizing the loss function** for simple models an anlytical solution may be available, but for most machine this requires an iterative optimization method to find the best model parameters

This process is repeated over a range of model complexities specified by hyperparameters. 

#### Machine Learning Model Tuning

The withheld testing data is retrieved and loss function (usually the **L2 norm** again) is calculated to summarize the error over the testing data

* this is repeated over over the range of specified hypparameters

* the model complexity / hyperparameters that minimize the loss function / error summary in testing is selected

This is known are model hypparameter tuning.

#### Machine Learning Model Overfit

More model complexity/flexibility than can be justified with the available data, data accuracy, frequency and coverage

* Model explains “idiosyncrasies” of the data, capturing data noise/error in the model

* High accuracy in training, but low accuracy in testing / real-world use away from training data cases – poor ability of the model to generalize


#### Workflow Goals

Learn the basics of machine learning training, tuning for model generalization while avoiding model overfit.

This includes:

* Demonstrate model training and tuning by hand with an interactive exercies

* Demonstrate the role of data error in leading to model overfit with complicated models

#### Getting Started

You will need to copy the following data files to your working directory.  They are available [here](https://github.com/GeostatsGuy/GeoDataSets):

* Tabular data - [Stochastic_1D_por_perm_demo.csv](https://github.com/GeostatsGuy/GeoDataSets/blob/master/Stochastic_1D_por_perm_demo.csv)
* Tabular data - [Random_Parabola.csv](https://github.com/GeostatsGuy/GeoDataSets/blob/master/Random_Parabola.csv)

These datasets are available in the folder: https://github.com/GeostatsGuy/GeoDataSets.


#### Import Required Packages

We will also need some standard packages. These should have been installed with Anaconda 3.

In [1]:
import geostatspy.GSLIB as GSLIB                       # GSLIB utilies, visualization and wrapper
import geostatspy.geostats as geostats                 # GSLIB methods convert to Python    

C:\Users\pm27995\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\pm27995\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\pm27995\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


We will also need some standard packages. These should have been installed with Anaconda 3.

In [2]:
%matplotlib inline
import os                                               # to set current working directory 
import sys                                              # supress output to screen for interactive variogram modeling
import io
import numpy as np                                      # arrays and matrix math
import pandas as pd                                     # DataFrames
import matplotlib.pyplot as plt                         # plotting
from sklearn.model_selection import train_test_split    # train and test split
from sklearn.metrics import mean_squared_error          # model error calculation
import scipy                                            # kernel density estimator for PDF plot
from matplotlib.pyplot import cm                        # color maps
from ipywidgets import interactive                      # widgets and interactivity
from ipywidgets import widgets                            
from ipywidgets import Layout
from ipywidgets import Label
from ipywidgets import VBox, HBox
import warnings
warnings.filterwarnings('ignore')                       # supress warnings

If you get a package import error, you may have to first install some of these packages. This can usually be accomplished by opening up a command window on Windows and then typing 'python -m pip install [package-name]'. More assistance is available with the respective package docs.  

#### Build the Interactive Dashboard

The following code:

* makes a random dataset, change the random number seed and number of data for a different dataset
* loops over polygonal fits of 1st-12th order, loops over mulitple realizations and calculates the average MSE and P10 and P90 vs. order
* calculates a specific model example
* plots the example model with training and testing data, the error distributions and the MSE envelopes vs. complexity

In [3]:
l = widgets.Text(value='                                       Machine Learning Overfit/Generalization Demo, Prof. Michael Pyrcz and John Eric McCarthy II, The University of Texas at Austin',
                 layout=Layout(width='950px', height='30px'))

n = widgets.IntSlider(min=15, max = 80, value=30, step = 1, description = 'n',orientation='horizontal', style = {'description_width': 'initial'}, continuous_update=False)
split = widgets.FloatSlider(min=0.05, max = .95, value=0.20, step = 0.05, description = 'Test %',orientation='horizontal',style = {'description_width': 'initial'}, continuous_update=False)
std = widgets.FloatSlider(min=0, max = 50, value=0, step = 1.0, description = 'Noise StDev',orientation='horizontal',style = {'description_width': 'initial'}, continuous_update=False)
degree = widgets.IntSlider(min=1, max = 12, value=1, step = 1, description = 'Model Order',orientation='horizontal', style = {'description_width': 'initial'}, continuous_update=False)

ui = widgets.HBox([n,split,std,degree],)
ui2 = widgets.VBox([l,ui],)

def run_plot(n,split,std,degree):
    seed = 13014; nreal = 20
    np.random.seed(seed) # seed the random number generator
    
    # make the datastet
    X_seq = np.linspace(0,20,100)
    X = np.random.rand(n)*20
    y = X*X + 50.0 # fit a parabola
    y = y + np.random.normal(loc = 0.0,scale=std,size=n) # add noise
    
    # calculate the MSE train and test over a range of complexity over multiple realizations of test/train split
    cdegrees = np.arange(1,13)
    cmse_train = np.zeros([len(cdegrees),nreal]); cmse_test = np.zeros([len(cdegrees),nreal])
    for j in range(0,nreal):
        for i, cdegree in enumerate(cdegrees):
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, random_state=seed+j)
            ccoefs = np.polyfit(X_train,y_train,cdegree)
            y_pred_train = np.polyval(ccoefs, X_train)
            y_pred_test = np.polyval(ccoefs, X_test)
            cmse_train[i,j] = mean_squared_error(y_train, y_pred_train)
            cmse_test[i,j] = mean_squared_error(y_test, y_pred_test)
    # summarize over the realizations
    cmse_train_avg = cmse_train.mean(axis=1)
    cmse_test_avg = cmse_test.mean(axis=1)
    cmse_train_high = np.percentile(cmse_train,q=90,axis=1)
    cmse_train_low = np.percentile(cmse_train,q=10,axis=1)        
    cmse_test_high = np.percentile(cmse_test,q=90,axis=1)
    cmse_test_low = np.percentile(cmse_test,q=10,axis=1)
    
#     cmse_train_high = np.amax(cmse_train,axis=1)
#     cmse_train_low = np.amin(cmse_train,axis=1)        
#     cmse_test_high = np.amax(cmse_test,axis=1)
#     cmse_test_low = np.amin(cmse_test,axis=1)
    
    # build the one model example to show
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, random_state=seed)
    coefs = np.polyfit(X_train,y_train,degree)    
    
    # calculate error
    error_seq = np.linspace(-100.0,100.0,100)
    error_train = np.polyval(coefs, X_train) - y_train
    #print(np.polyval(coefs, X_train))
    #print('truth')
    #print(X_train)
    error_test = np.polyval(coefs, X_test) - y_test
    
    mse_train = mean_squared_error(y_train, np.polyval(coefs, X_train))
    mse_test = mean_squared_error(y_test, np.polyval(coefs, X_test))
    
    error_train_std = np.std(error_train)
    error_test_std = np.std(error_test)
    
    kde_error_train = scipy.stats.gaussian_kde(error_train)
    kde_error_test = scipy.stats.gaussian_kde(error_test)
    
    plt.subplot(131)
    plt.plot(X_seq, np.polyval(coefs, X_seq), color="black")
    plt.title("Polynomial Model of Degree = "+str(degree))
    plt.scatter(X_train,y_train,c ="red",alpha=0.2,edgecolors="black")
    plt.scatter(X_test,y_test,c ="blue",alpha=0.2,edgecolors="black")
    plt.ylim([0,500]); plt.xlim([0,20]); plt.grid()
    plt.xlabel('Porosity (%)'); plt.ylabel('Permeability (mD)')
    
    plt.subplot(132)
    plt.hist(error_train, facecolor='red',bins=np.linspace(-50.0,50.0,10),alpha=0.2,density=True,edgecolor='black',label='Train')
    plt.hist(error_test, facecolor='blue',bins=np.linspace(-50.0,50.0,10),alpha=0.2,density=True,edgecolor='black',label='Test')
    #plt.plot(error_seq,kde_error_train(error_seq),lw=2,label='Train',c='red')
    #plt.plot(error_seq,kde_error_test(error_seq),lw=2,label='Test',c='blue')   
    plt.xlim([-55.0,55.0]); plt.ylim([0,0.1])
    plt.xlabel('Model Error'); plt.ylabel('Frequency'); plt.title('Training and Testing Error, Model of Degree = '+str((degree)))
    plt.legend(loc='upper left')
    plt.grid(True)
    
    plt.subplot(133); ax = plt.gca()
    plt.plot(cdegrees,cmse_train_avg,lw=2,label='Train',c='red')
    ax.fill_between(cdegrees,cmse_train_high,cmse_train_low,facecolor='red',alpha=0.05)
 
    plt.plot(cdegrees,cmse_test_avg,lw=2,label='Test',c='blue')  
    ax.fill_between(cdegrees,cmse_test_high,cmse_test_low,facecolor='blue',alpha=0.05)
    plt.xlim([1,12]); plt.yscale('log'); plt.ylim([0.0000001,10000])
    plt.xlabel('Complexity - Polynomial Order'); plt.ylabel('Mean Square Error'); plt.title('Training and Testing Error vs. Model Complexity')
    plt.legend(loc='upper left')
    plt.grid(True)
    
    plt.plot([degree,degree],[.0000001,100000],c = 'black',linewidth=3,alpha = 0.8)
    
    plt.subplots_adjust(left=0.0, bottom=0.0, right=2.0, top=1.6, wspace=0.2, hspace=0.3)
    plt.show()
    
# connect the function to make the samples and plot to the widgets    
interactive_plot = widgets.interactive_output(run_plot, {'n':n,'split':split,'std':std,'degree':degree})
interactive_plot.clear_output(wait = True)               # reduce flickering by delaying plot updating


### Interactive Predictive Machine Learning Overfitting Demonstation 

#### Michael Pyrcz, Associate Professor, The University of Texas at Austin 

Change the number of sample data, train/test split and the data noise and observe overfit! Change the model order to observe a specific model example.

### The Inputs

* **n** - number of data
* **Test %** - percentage of sample data withheld as testing data
* **Noise StDev** - standard deviation of random Gaussian error added to the data
* **Model Order** - the order of the 

In [6]:
display(ui2, interactive_plot)                           # display the interactive plot

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<Figure size 432x288 with 3 Axes>', 'i…

#### Comments

This was a basic demonstration of machine learning model training and tuning, model generalization and complexity. I have many other demonstrations and even basics of working with DataFrames, ndarrays, univariate statistics, plotting data, declustering, data transformations and many other workflows available at https://github.com/GeostatsGuy/PythonNumericalDemos and https://github.com/GeostatsGuy/GeostatsPy. 
  
#### The Author:

### Michael Pyrcz, Associate Professor, University of Texas at Austin 
*Novel Data Analytics, Geostatistics and Machine Learning Subsurface Solutions*

With over 17 years of experience in subsurface consulting, research and development, Michael has returned to academia driven by his passion for teaching and enthusiasm for enhancing engineers' and geoscientists' impact in subsurface resource development. 

For more about Michael check out these links:

#### [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)

#### Want to Work Together?

I hope this content is helpful to those that want to learn more about subsurface modeling, data analytics and machine learning. Students and working professionals are welcome to participate.

* Want to invite me to visit your company for training, mentoring, project review, workflow design and / or consulting? I'd be happy to drop by and work with you! 

* Interested in partnering, supporting my graduate student research or my Subsurface Data Analytics and Machine Learning consortium (co-PIs including Profs. Foster, Torres-Verdin and van Oort)? My research combines data analytics, stochastic modeling and machine learning theory with practice to develop novel methods and workflows to add value. We are solving challenging subsurface problems!

* I can be reached at mpyrcz@austin.utexas.edu.

I'm always happy to discuss,

*Michael*

Michael Pyrcz, Ph.D., P.Eng. Associate Professor The Hildebrand Department of Petroleum and Geosystems Engineering, Bureau of Economic Geology, The Jackson School of Geosciences, The University of Texas at Austin

#### More Resources Available at: [Twitter](https://twitter.com/geostatsguy) | [GitHub](https://github.com/GeostatsGuy) | [Website](http://michaelpyrcz.com) | [GoogleScholar](https://scholar.google.com/citations?user=QVZ20eQAAAAJ&hl=en&oi=ao) | [Book](https://www.amazon.com/Geostatistical-Reservoir-Modeling-Michael-Pyrcz/dp/0199731446) | [YouTube](https://www.youtube.com/channel/UCLqEr-xV-ceHdXXXrTId5ig)  | [LinkedIn](https://www.linkedin.com/in/michael-pyrcz-61a648a1)  
  